<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 3
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10
## <center> Прогнозирование задержек вылетов

Ваша задача – побить как минимум 2 бенчмарка в [соревновании](https://www.kaggle.com/c/flight-delays-spring-2018) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен второй – с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит градиентный бустинг. Скорее всего Xgboost, но тут у нас еще немало категориальных признаков...

<img src='../../img/xgboost_meme.jpg' width=40% />

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDRegressor
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix, hstack

In [2]:
train = pd.read_csv('../../data/flight_delays_train.csv')
test = pd.read_csv('../../data/flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


In [5]:
y_train = train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

In [6]:
all_data = pd.concat([train.drop('dep_delayed_15min', axis=1), test])

In [7]:
all_data.shape, test.shape, train.shape

((200000, 8), (100000, 8), (100000, 9))

In [8]:
idx_split = train.shape[0]

In [9]:
all_data['route'] = all_data['Origin'] + all_data['Dest']

In [10]:
all_data.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,route
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,ATLDFW
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,PITMCO
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,RDUCLE
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,DENMEM
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,MDWOMA


In [11]:
all_data['route_num'] = LabelEncoder().fit_transform(all_data['route'])
all_data['Month_num'] = LabelEncoder().fit_transform(all_data['Month'])
all_data['DayofMonth_num'] = LabelEncoder().fit_transform(all_data['DayofMonth'])
all_data['DayOfWeek_num'] = LabelEncoder().fit_transform(all_data['DayOfWeek'])
all_data['UniqueCarrier_num'] = LabelEncoder().fit_transform(all_data['UniqueCarrier'])

In [12]:
#to_scr = ['route_num', 'Month_num', 'DayofMonth_num', 'DayOfWeek_num', 'UniqueCarrier_num', 'Distance', 'DepTime']
to_scr = ['Distance', 'DepTime']

In [13]:
scaler = StandardScaler()
for i in to_scr:
    all_data[i] = scaler.fit_transform(all_data[i].values.reshape(-1, 1))

In [14]:
all_data.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,route,route_num,Month_num,DayofMonth_num,DayOfWeek_num,UniqueCarrier_num
0,c-8,c-21,c-7,1.240975,AA,ATL,DFW,0.010082,ATLDFW,171,10,13,6,1
1,c-4,c-20,c-3,0.434238,US,PIT,MCO,0.189359,PITMCO,3986,6,12,2,19
2,c-9,c-2,c-5,0.170898,XE,RDU,CLE,-0.545323,RDUCLE,4091,11,11,4,21
3,c-11,c-25,c-6,-0.679729,OO,DEN,MEM,0.256148,DENMEM,1304,2,17,5,16
4,c-10,c-7,c-6,1.019436,WN,MDW,OMA,-0.533019,MDWOMA,2979,1,28,5,20


In [15]:
ohe = OneHotEncoder()

In [16]:
#Добавить Scaler
route = ohe.fit_transform(all_data['route_num'].reshape(-1, 1))
Month = ohe.fit_transform(all_data['Month_num'].reshape(-1, 1))
DayofMonth = ohe.fit_transform(all_data['DayofMonth_num'].reshape(-1, 1))
DayOfWeek = ohe.fit_transform(all_data['DayOfWeek_num'].reshape(-1, 1))
UniqueCarrier = ohe.fit_transform(all_data['UniqueCarrier_num'].reshape(-1, 1))

In [17]:
scaler.fit(all_data['Distance'].values.reshape(-1, 1))

all_data['Distance_scaled'] = scaler.transform(all_data['Distance'].values.reshape(-1, 1))

In [18]:
#scaler.fit(all_data['Distance'].values.reshape(-1, 1))

all_data['DepTime_scaled'] = scaler.fit_transform(all_data['DepTime'].values.reshape(-1, 1))

In [19]:
#to_scr = ['route_num', 'Month_num', 'DayofMonth_num', 'DayOfWeek_num', 'UniqueCarrier_num', 'Distance', 'DepTime']

In [20]:
full_sparse = csr_matrix(hstack([route, Month]))

In [21]:
full_sparse = csr_matrix(hstack([full_sparse, DayofMonth]))

In [22]:
full_sparse = csr_matrix(hstack([full_sparse, DayOfWeek]))

In [23]:
full_sparse = csr_matrix(hstack([full_sparse, UniqueCarrier]))

In [24]:
full_sparse = csr_matrix(hstack([full_sparse, all_data['Distance_scaled'].values.reshape(-1, 1)]))
#Признаки Distance и  DepTime брались без изменений

In [25]:
full_sparse = csr_matrix(hstack([full_sparse, all_data['DepTime_scaled'].values.reshape(-1, 1)]))

In [26]:
full_sparse

<200000x5123 sparse matrix of type '<class 'numpy.float64'>'
	with 1400000 stored elements in Compressed Sparse Row format>

In [27]:
X_train = full_sparse[:idx_split, :]

In [28]:
X_train

<100000x5123 sparse matrix of type '<class 'numpy.float64'>'
	with 700000 stored elements in Compressed Sparse Row format>

In [158]:
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_train, y_train, test_size=0.3, random_state = 17)

-----------

## Logit

In [169]:
logit = LogisticRegression(random_state=17)

In [170]:
lr_params = {'C': np.linspace(1, 4, num=10)}

In [171]:
kf = KFold(n_splits=3, shuffle=True, random_state=17)

In [172]:
lrc = LogisticRegressionCV(random_state=17, scoring='roc_auc', n_jobs=2, cv=kf)

In [173]:
%%time
lrc.fit(X_train_train, y_train_train)

CPU times: user 1.18 s, sys: 83.7 ms, total: 1.26 s
Wall time: 12.8 s


LogisticRegressionCV(Cs=10, class_weight=None,
           cv=KFold(n_splits=3, random_state=17, shuffle=True), dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=2, penalty='l2', random_state=17,
           refit=True, scoring='roc_auc', solver='lbfgs', tol=0.0001,
           verbose=0)

In [174]:
print('C = ', lrc.C_[0])

C =  0.046415888336127774


In [175]:
lrc.scores_

{1: array([[0.69279833, 0.69797707, 0.70391288, 0.70812576, 0.70695421,
         0.6881046 , 0.67237204, 0.66881257, 0.66597038, 0.66645911],
        [0.68153057, 0.68642404, 0.6915032 , 0.69460515, 0.6931045 ,
         0.67441973, 0.65892162, 0.6562407 , 0.65345917, 0.65383572],
        [0.6891697 , 0.69438995, 0.6993706 , 0.7017661 , 0.69804024,
         0.67616831, 0.65874526, 0.65532539, 0.65203745, 0.652618  ]])}

In [176]:
lr = LogisticRegression(C=lrc.C_[0])
cvs = cross_val_score(lr, X_train_test, y_train_test, cv=kf, n_jobs=2, scoring='roc_auc')

In [177]:
round(cvs.mean(), 4)

0.6909

In [178]:
X_test = full_sparse[idx_split:,:]

In [179]:
lr.fit(X_train_train, y_train_train)
y_pred_lr = lr.predict_proba(X_train_test)[:, 1]
#logit_test_pred = lr.predict_proba(X_test)[:, 1]

In [180]:
roc_auc_score(y_train_test, y_pred_lr)

0.6980132458978102

----------------

## XGB

In [144]:
#xgb = XGBClassifier(eta = 0.1, silent = 1, random_state=17, objective = "binary:logistic", max_depth=7, nthread=4)
xgb = XGBClassifier(eta = 0.1, random_state=17, 
                    objective = "binary:logistic", max_depth=13, n_estimators=500, nthread=4)
#ax_depth = 20, n_estimators = 10 

In [145]:
%%time
xgb.fit(X_train_train, y_train_train)

CPU times: user 1min 23s, sys: 1.49 s, total: 1min 24s
Wall time: 24.7 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=13, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=17, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [146]:
y_pred_lr = xgb.predict_proba(X_train_test)[:, 1]

In [147]:
round(roc_auc_score(y_train_test, y_pred_lr), 4)

0.7344

In [148]:
xgb_params = {'max_depth': [12,13,14,15,16]
              #, 'n_estimators': [5,10,15,20]
             }

In [149]:
gs = GridSearchCV(estimator=xgb, param_grid=xgb_params, cv=kf, n_jobs=4)

In [150]:
%%time
gs.fit(X_train_train, y_train_train)

/zmeyoff/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/zmeyoff/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/zmeyoff/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/zmeyoff/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

CPU times: user 1min 26s, sys: 1.84 s, total: 1min 28s
Wall time: 7min 31s


GridSearchCV(cv=KFold(n_splits=3, random_state=17, shuffle=True),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=13, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=17, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'max_depth': [12, 13, 14, 15, 16]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [151]:
gs.best_score_

0.8179125

In [152]:
gs.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=12, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=17, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [153]:
gs.best_params_

{'max_depth': 12}

In [154]:
gs.cv_results_['mean_train_score']

array([0.85616251, 0.8615375 , 0.86825626, 0.87495625, 0.88111876])

In [159]:
xgb = XGBClassifier(eta = 0.1, random_state=17, 
                    objective = "binary:logistic", max_depth=12, n_estimators=500, nthread=4)

In [160]:
%%time
cvs = cross_val_score(xgb, X_train_test, y_train_test, cv=kf, n_jobs=4, scoring='roc_auc')

CPU times: user 85.2 ms, sys: 43.6 ms, total: 129 ms
Wall time: 33.3 s


In [161]:
round(cvs.mean(), 4)

0.7055

### 0.6989

------------------

## SGD

In [161]:
sgd= SGDRegressor(random_state=17)

In [162]:
%%time
cvs = cross_val_score(sgd, X_train_test, y_train_test, cv=kf, n_jobs=2, scoring='roc_auc')

Wall time: 1.81 s


In [163]:
cvs

array([0.6899151 , 0.67853903, 0.69615381])

In [164]:
sgd.fit(X_train_train, y_train_train)
y_pred_sgd = sgd.predict(X_train_test)

In [165]:
roc_auc_score(y_train_test, y_pred_sgd)

0.6950724168521942

In [166]:
w1 = 1.6

In [167]:
y_pred = w1*y_pred_lr+(1-w1)*y_pred_sgd

In [168]:
y_pred

array([0.07950143, 0.06907586, 0.22016036, ..., 0.09218836, 0.07903194,
       0.34887855])

In [169]:
roc_auc_score(y_train_test, y_pred)

0.69991339956501

In [164]:
%%time
xgb.fit(X_train_train, y_train_train)

CPU times: user 1min 12s, sys: 1.43 s, total: 1min 13s
Wall time: 22.4 s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=12, min_child_weight=1, missing=None,
       n_estimators=500, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=17, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [165]:
y_pred_xgb = xgb.predict_proba(X_train_test)[:, 1]

In [198]:
w1 = 0.2

In [199]:
y_pred = w1*y_pred_lr+(1-w1)*y_pred_xgb

In [200]:
roc_auc_score(y_train_test, y_pred)

0.7313308951925273

In [212]:
max_score = 0
best_param = 0
for i in np.linspace(0.001, 1, 100):
    w1 = i
    y_pred = w1*y_pred_lr+(1-w1)*y_pred_xgb
    if max_score< roc_auc_score(y_train_test, y_pred):
        max_score = roc_auc_score(y_train_test, y_pred)
        best_param = w1
        #print('best score=', max_score, '\tbest param=', best_param)
print(best_param, max_score)        

0.13218181818181818 0.7314485671918033


-----

In [213]:
w1 = 0.13

In [214]:
%%time
lr.fit(X_train, y_train)
logit_test_pred_lr = lr.predict_proba(X_test)[:, 1]
#logit_test_pred_sgd = sgd.predict(X_test)
logit_test_pred_xgb = xgb.predict_proba(X_test)[:, 1]

logit_test_pred = w1*logit_test_pred_lr+(1-w1)*logit_test_pred_xgb


pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat_zm_002.csv', index_label='id', header=True)

CPU times: user 6.72 s, sys: 96 ms, total: 6.81 s
Wall time: 2.4 s


-------------

Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [5]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_part = scaler.fit_transform(X_train_part)
X_valid = scaler.transform(X_valid)

In [6]:
logit = LogisticRegression()

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.6795691465352607

In [7]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logit.fit(X_train_scaled, y_train)
logit_test_pred = logit.predict_proba(X_test_scaled)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)

Второй бенчмарк, представленный в рейтинге соревнования, был получен так:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, вы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!